### 텍스트마이닝
- 대량의 텍스트 데이터에서 의미있는 정보를 추출하는 작업
- 텍스트 분류, 감성분석, 키워드 분석, 자연어 처리 등

### 머신러닝 프로세스 for Text Mining 
1. 문제정의
2. 데이터 수집
3. 데이터 전처리
    - 토큰화 : 텍스트 데이터를 쪼개는 작업 ( 필수 )
        1. 단어 중심
        2. 글자 중심
        3. n-gram
        4. 형태소 중심(konlpy)
    - 수치화 : 글자를 숫자 형태로 변경하는 작업 (필수)
        1. 빈도수
        2. 라벨/원핫인코딩
        3. BOW(bag of words)
        4. 워드임베딩(딥러닝)
    - 불용어 제거(stop word) : 사용하지 않은 단어처리
    - 오탈자 제거, 띄어쓰기 교정, 이모티콘 수정
    - 정제, 정규화, 어간추출, 표제어추출
4. 탐색적 데이터 분석(EDA)
5. 모델 선택
6. 모델 학습 및 하이퍼 파라미터 튜닝 
7. 평가 

In [71]:
from sklearn.feature_extraction.text import CountVectorizer # BOW 규현 클래스

### 간단 데이터 전처리 실습

In [72]:
sample_test=['오늘 점심은 뭘까',
            '오늘 간식은 킷캣',
            '내일 간식은 커피']

In [73]:
sample_cvt = CountVectorizer()

In [74]:
sample_cvt.fit(sample_test) # 토큰화 및 단어사전 구축

CountVectorizer()

In [75]:
sample_cvt.vocabulary_

{'오늘': 3, '점심은': 4, '뭘까': 2, '간식은': 0, '킷캣': 6, '내일': 1, '커피': 5}

In [76]:
sample_cvt.transform(sample_test) # 새로운 데이터 크기 조정

<3x7 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [77]:
sample_cvt.fit_transform(sample_test)

<3x7 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

- fit
    - 스케일링에 사용할 평균 및 표준편차를 계산 ( 단지 계싼만)
- transform
    - fit로 계산된 평균과 표준편차를 사용하여 데이터의 크기 조정
- fit_transform
    - 위의 두가지를 동시에 진행
    - train dataset에서만 사용하는걸 권장

In [78]:
# 수치데이터 만듬
sample_transformed = sample_cvt.transform(sample_test)

In [79]:
sample_transformed.toarray() # 압축된 포맷을 풀어서 확인 (일반적으로는 쓰지 않음)

array([[0, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 0, 1, 0]], dtype=int64)

### 감성분석 - 네이버 영화리뷰 데이터

#### 데이터셋 다운로드
- 네이버 영화리뷰 데이터
- http://github.com/e9t/nsmc

In [80]:
import pandas as pd

In [81]:
train = pd.read_csv("./data2/ratings_train.txt",delimiter='\t')
test = pd.read_csv("./data2/ratings_test.txt",delimiter='\t')

In [82]:
train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [83]:
# 결측치 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [84]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [85]:
train.dropna(inplace=True) #결측치행 다 삭제

In [86]:
test.dropna(inplace=True)

In [87]:
X_train =train['document']
y_train =train['label']
X_test =test['document']
y_test =test['label']
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((149995,), (149995,), (49997,), (49997,))

#### 토큰화, 수치화 적용

In [88]:
movie_cvt = CountVectorizer()

In [89]:
X_train_transformed = movie_cvt.fit_transform(X_train)
X_test_transformed = movie_cvt.transform(X_test)

In [90]:
X_train_transformed.shape, X_test_transformed.shape

((149995, 293366), (49997, 293366))

#### 모델링

In [91]:
from sklearn.linear_model import LogisticRegression


In [92]:
import warnings
warnings.filterwarnings('ignore')

In [93]:
movie_logi = LogisticRegression()

In [94]:
movie_logi.fit(X_train_transformed,y_train)

LogisticRegression()

In [95]:
movie_logi.score(X_test_transformed,y_test)

0.814428865731944

#### 활용

In [96]:
sample_text=['진입장벽이 높으나 기존 세계관 이해만 잘 되었다면 상당히 재밌게 볼 수 있을듯!',
            '감동적이네요.그치만 사투리연기는 아무나해선 안됨ㅠㅠ',
            '전편에 밑도 끝도 없이 시작했던 스토리를 설명하는 초반 15분 시퀀스는 이 영화의 백미! 결말이 너무나 기다려 진다',
            '중국풍나는 재단에서 정이 훅 떨어졌어요',
            '왜이렇게 평점이 낮은지 모르겠네..',
            '기대한것보다 기대이하너무 기대하고보진마세요',
            '본격적인 마블의 하락세가 시작되었구나',
            '대박재밌습니다 보기전에 완다비전 꼭 보구요',
            '너무 너무 화가 난다. 산드라 오 피벨 스튜어트 둘다 좋아하는 배우인데 마음이 아픔 오리엔탈리즘의 정수임.. 기분 진짜 나쁘다,',
            '제 개인의 기준으로 보면 너무 재밌게 보았습니다. 마블 영화에 대단한 스토리나 철학을 기대하지 않으니까요.']

In [97]:
# 정형화 시키기
sample_transformed = movie_cvt.transform(sample_text)

In [98]:
movie_logi.predict_proba(sample_transformed)

array([[0.23964391, 0.76035609],
       [0.12760069, 0.87239931],
       [0.9166007 , 0.0833993 ],
       [0.4440187 , 0.5559813 ],
       [0.2857108 , 0.7142892 ],
       [0.52735185, 0.47264815],
       [0.47123833, 0.52876167],
       [0.47718278, 0.52281722],
       [0.68753215, 0.31246785],
       [0.083569  , 0.916431  ]])

#### 학습결과 시각화 
- 지금 선형모델은 293366개의 단어마다 가중치를 학습
- 가중치 : 각 단어가 긍정/부정에 영향을 미치는 정도
- 해당 단어가 긍정이나 부정 등 특정 감성에 많이 등장을 하면 양수로 커지거나, 음수로 작아진다.
    - 애매하면 가중치가 0에 가까워짐

In [99]:
movie_logi.coef_.shape

(1, 293366)

In [100]:
word_dic = movie_cvt.vocabulary_
word_dic

{'더빙': 71119,
 '진짜': 246232,
 '짜증나네요': 248358,
 '목소리': 99567,
 '포스터보고': 273335,
 '초딩영화줄': 255126,
 '오버연기조차': 190112,
 '가볍지': 16352,
 '않구나': 167602,
 '너무재밓었다그래서보는것을추천한다': 57394,
 '교도소': 33783,
 '이야기구먼': 208071,
 '솔직히': 145795,
 '재미는': 222295,
 '없다': 177352,
 '평점': 271982,
 '조정': 234711,
 '사이몬페그의': 133947,
 '익살스런': 210575,
 '연기가': 181881,
 '돋보였던': 74028,
 '영화': 185057,
 '스파이더맨에서': 150442,
 '늙어보이기만': 63331,
 '했던': 283593,
 '커스틴': 261359,
 '던스트가': 71680,
 '너무나도': 56734,
 '이뻐보였다': 207059,
 '걸음마': 25696,
 '3세부터': 5282,
 '초등학교': 254957,
 '1학년생인': 3039,
 '8살용영화': 7674,
 'ㅋㅋㅋ': 13347,
 '별반개도': 117898,
 '아까움': 160393,
 '원작의': 198609,
 '긴장감을': 43582,
 '제대로': 232430,
 '살려내지못했다': 134933,
 '반개도': 110754,
 '아깝다': 160496,
 '욕나온다': 194727,
 '이응경': 209070,
 '길용우': 43801,
 '연기생활이몇년인지': 182309,
 '정말': 230402,
 '발로해도': 112228,
 '그것보단': 36866,
 '낫겟다': 53960,
 '납치': 53930,
 '감금만반복반복': 18635,
 '이드라마는': 204973,
 '가족도없다': 17264,
 '연기못하는사람만모엿네': 182260,
 '액션이': 171312,
 '없는데도': 177305,
 '재미': 222156,
 '있는': 2148

In [101]:
result = pd.DataFrame([word_dic.keys(),word_dic.values()]).T
result

,0,1
0,더빙,71119
1,진짜,246232
2,짜증나네요,248358
3,목소리,99567
4,포스터보고,273335
...,...,...
293361,더블은,71115
293362,뭔죄인가,105744
293363,거들먹거리고,24486
293364,혼혈은,287413


In [102]:
result_sorted=result.sort_values(by=1)
result_sorted

,0,1
126724,00,0
85452,000,1
123217,0000000000000000,2
245647,00000000000000000000000000000을달라,3
55188,000000000000000001개짜리,4
...,...,...
240192,힣히히헤ㅎ,293361
278346,盧미오,293362
187622,綠林의,293363
220971,不好,293364


In [103]:
result_sorted['weight']=movie_logi.coef_.reshape(-1)
result_sorted.head()

,0,1,weight
126724,00,0,0.526995
85452,000,1,0.001948
123217,0000000000000000,2,-0.355767
245647,00000000000000000000000000000을달라,3,-0.355767
55188,000000000000000001개짜리,4,-0.147017


In [104]:
final_result=result_sorted.sort_values(by='weight')
final_result.head()

,0,1,weight
1982,최악의,256927,-4.465697
1115,최악,256888,-3.889746
3490,쓰레기영화,159352,-3.752756
554,졸작,235972,-3.559744
161,재미없다,222651,-3.391995


In [105]:
final_result.tail()

,0,1,weight
921,여운이,180158,2.908326
220,좋았어요,237604,2.915229
7643,10점준다,1057,2.980369
2144,최고다,256234,3.052881
409,재미있어요,223318,3.298084


#### TF_IDF 수치화
- 전체 문서에서는 자주 등장하지 않지만, 특정문서에서 여러번 등장하는 단어를 수치화하는 방법

In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [107]:
tf_idf_vct = TfidfVectorizer()

In [108]:
X_train_transformed_tf=tf_idf_vct.fit_transform(X_train)

In [109]:
X_train_transformed_tf

<149995x293366 sparse matrix of type '<class 'numpy.float64'>'
	with 1074805 stored elements in Compressed Sparse Row format>

In [110]:
movie_logi2 = LogisticRegression()
movie_logi2.fit(X_train_transformed_tf, y_train)

LogisticRegression()

In [111]:
X_test_transformed_tf =tf_idf_vct.transform(X_test)
movie_logi2.score(X_test_transformed_tf,y_test)

0.8127287637258236

#### Pipe line
- 일련의 머신러닝 과정들 처리해줌

In [112]:
from sklearn.pipeline import make_pipeline

In [122]:
pipe_model = make_pipeline(TfidfVectorizer(),LogisticRegression())

In [123]:
pipe_model.fit(X_train,y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [124]:
pipe_model.score(X_test,y_test)

0.8127287637258236

#### 하이퍼 파라미터 튜닝
- TfidfVectorizer
    - min_df : 전체 문서에서 최소 등장해야하는 수
    - max_df : 전체 문서에서 최대 등장해야하는 수
    - n_gram : 단어 토큰을 여러개 연결하는 파라미터
- LogisticRegression
    - C: 규제 강도

#### Grid Search

In [125]:
from sklearn.model_selection import GridSearchCV

In [117]:
grid_params={
    'tfidfvectorizer__min_df' : [5,15,25],
    'tfidfvectorizer__max_df' : [50000,100000],
    'tfidfvectorizer_ngram_range':[(1,1),(1,2),(2,2)],
    'logisticregression__C' : [0.01,0.1,1,10,100]
}

In [126]:
grid_params ={
    'tfidfvectorizer__min_df' : [5,15,25],
    'tfidfvectorizer__max_df' : [50000,100000],
    'tfidfvectorizer__ngram_range' : [(1,1),(1,2),(2,2)],
    'logisticregression__C' : [0.01,0.1,1,10,100]
}

In [127]:
grid = GridSearchCV(pipe_model,grid_params,cv=3,verbose=3,n_jobs=-1)

In [128]:
grid.fit(X_train,y_train)

Fitting 3 folds for each of 90 candidates, totalling 270 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                                       ('logisticregression',
                                        LogisticRegression())]),
             n_jobs=-1,
             param_grid={'logisticregression__C': [0.01, 0.1, 1, 10, 100],
                         'tfidfvectorizer__max_df': [50000, 100000],
                         'tfidfvectorizer__min_df': [5, 15, 25],
                         'tfidfvectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (2, 2)]},
             verbose=3)

In [129]:
grid.best_params_

{'logisticregression__C': 1,
 'tfidfvectorizer__max_df': 50000,
 'tfidfvectorizer__min_df': 5,
 'tfidfvectorizer__ngram_range': (1, 2)}

In [130]:
grid.best_score_

0.799433297758319

In [131]:
best_model = grid.best_estimator_

In [132]:
best_model.score(X_test,y_test)

0.8077484649078944

In [133]:
!pip install konlpy

     --------------------------------------- 19.4/19.4 MB 10.7 MB/s eta 0:00:00
     ------------------------------------- 345.2/345.2 kB 10.8 MB/s eta 0:00:00


In [138]:
from konlpy.tag import Okt

In [139]:
okt=Okt()

In [140]:
okt.nouns("아버지가 방에 들어 가신다.")

['아버지', '방']

In [141]:
!pip install JPype1-1.3.0-cp37-cp37m-win_amd64.whl

ERROR: JPype1-1.3.0-cp37-cp37m-win_amd64.whl is not a supported wheel on this platform.
